In [1]:
import utils
import json
from data_utils import TextAudioLoader, TextAudioCollate, DistributedBucketSampler
import torch

/home/husein/tf2/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('config/base.json') as fopen:
    config = json.load(fopen)

hps = utils.HParams(**config)
hps.data.validation_files, hps.data

('/home/husein/speech-bahasa/vits-test-set.txt',
 {'training_files': '/home/husein/speech-bahasa/vits-train-set.txt', 'validation_files': '/home/husein/speech-bahasa/vits-test-set.txt', 'text_cleaners': [''], 'max_wav_value': 32768.0, 'sampling_rate': 22050, 'filter_length': 1024, 'hop_length': 256, 'win_length': 1024, 'n_mel_channels': 80, 'mel_fmin': 0.0, 'mel_fmax': None, 'add_blank': True, 'n_speakers': 0, 'cleaned_text': True})

In [3]:
train_dataset = TextAudioLoader(hps.data.validation_files, hps.data)

In [4]:
len(train_dataset)

1974

In [5]:
text, spec, wav = train_dataset[0]

In [6]:
text.shape, spec.shape, wav.shape

(torch.Size([217]), torch.Size([513, 615]), torch.Size([1, 157552]))

In [7]:
collate = TextAudioCollate()

In [8]:
batch = [train_dataset[i] for i in range(5)]

In [9]:
collate(batch)

(tensor([[ 0, 14,  0,  ...,  0,  9,  0],
         [ 0, 24,  0,  ...,  0,  0,  0],
         [ 0, 20,  0,  ...,  0,  0,  0],
         [ 0, 29,  0,  ...,  0,  0,  0],
         [ 0, 33,  0,  ...,  0,  0,  0]]),
 tensor([297, 219, 217, 209, 117]),
 tensor([[[0.0088, 0.0079, 0.0111,  ..., 0.0151, 0.0134, 0.0125],
          [0.0054, 0.0027, 0.0067,  ..., 0.0076, 0.0067, 0.0061],
          [0.0025, 0.0025, 0.0016,  ..., 0.0010, 0.0012, 0.0011],
          ...,
          [0.0011, 0.0010, 0.0011,  ..., 0.0011, 0.0010, 0.0010],
          [0.0011, 0.0011, 0.0011,  ..., 0.0012, 0.0012, 0.0012],
          [0.0012, 0.0010, 0.0011,  ..., 0.0010, 0.0011, 0.0014]],
 
         [[0.0103, 0.0156, 0.0216,  ..., 0.0000, 0.0000, 0.0000],
          [0.0048, 0.0077, 0.0144,  ..., 0.0000, 0.0000, 0.0000],
          [0.0013, 0.0040, 0.0057,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0010, 0.0010, 0.0010,  ..., 0.0000, 0.0000, 0.0000],
          [0.0010, 0.0010, 0.0010,  ..., 0.0000, 0.0000, 0.0000]

In [15]:
train_sampler = DistributedBucketSampler(
        train_dataset,
        4,
        [32, 300, 400, 500, 600, 700, 800, 900, 1000],
        shuffle=True, num_replicas = 1, rank = 0)

In [25]:
iterator = iter(train_sampler)

In [24]:
len(train_sampler)

495

In [29]:
batch = [train_dataset[i] for i in next(iterator)]
batch

/home/husein/tf2/lib/python3.6/site-packages/torch/functional.py:573: UserWarning: stft will soon require the return_complex parameter be given for real inputs, and will further require that return_complex=True in a future PyTorch release. (Triggered internally at  ../aten/src/ATen/native/SpectralOps.cpp:659.)
  normalized, onesided, return_complex)


[(tensor([ 0, 29,  0, 44,  0, 53,  0, 64,  0, 40,  0, 50,  0, 48,  0, 59,  0, 13,
           0, 48,  0, 53,  0, 48,  0, 13,  0, 43,  0, 48,  0, 50,  0, 54,  0, 53,
           0, 45,  0, 48,  0, 57,  0, 52,  0, 40,  0, 58,  0, 48,  0, 13,  0, 59,
           0, 44,  0, 51,  0, 40,  0, 47,  0, 13,  0, 58,  0, 40,  0, 52,  0, 55,
           0, 40,  0, 48,  0, 13,  0, 43,  0, 48,  0, 13,  0, 32,  0, 48,  0, 53,
           0, 46,  0, 40,  0, 55,  0, 60,  0, 57,  0, 40,  0, 13,  0,  9,  0]),
  tensor([[0.0112, 0.0111, 0.0209,  ..., 0.0162, 0.0129, 0.0114],
          [0.0069, 0.0027, 0.0147,  ..., 0.0099, 0.0056, 0.0058],
          [0.0018, 0.0043, 0.0054,  ..., 0.0046, 0.0023, 0.0018],
          ...,
          [0.0011, 0.0010, 0.0010,  ..., 0.0012, 0.0010, 0.0010],
          [0.0011, 0.0011, 0.0010,  ..., 0.0017, 0.0011, 0.0011],
          [0.0014, 0.0012, 0.0010,  ..., 0.0018, 0.0010, 0.0012]]),
  tensor([[-3.0518e-05, -3.0518e-05, -6.1035e-05,  ...,  0.0000e+00,
            0.0000e+00, -6.1